## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 13th Oct. 20th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-15-2024,Condo/Co-op,1750 NE 191st St Unit 725-4,Miami,FL,33179.0,205000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1750-NE-191st-...,MARMLS,A11677256,N,Y,25.949213,-80.16741


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
34,PAST SALE,2024-10-21,Condo/Co-op,4590 NW 79th Ave Unit 2D,Doral,FL,33166.0,270000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/4590-NW-79th-A...,MARMLS,A11610586,N,Y,25.816384,-80.326601
74,PAST SALE,2024-10-18,Condo/Co-op,17375 Collins Ave #1201,Sunny Isles Beach,FL,33160.0,885000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11619031,N,Y,25.921232,-80.122311
53,PAST SALE,2024-10-18,Condo/Co-op,820 SW 105th Ave #607,Miami,FL,33174.0,293000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/820-SW-105th-A...,MARMLS,A11608585,N,Y,25.760351,-80.366017
23,PAST SALE,2024-10-18,Condo/Co-op,7705 Camino Real Unit B-113,Miami,FL,33143.0,310000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7705-Camino-Re...,MARMLS,A11636340,N,Y,25.694293,-80.318588
91,PAST SALE,2024-10-18,Condo/Co-op,7928 East Dr #1601,North Bay Village,FL,33141.0,1388000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Bay-Village/79...,MARMLS,A11577821,N,Y,25.852182,-80.157834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,PAST SALE,2024-10-15,Condo/Co-op,20335 W Country Club Dr #1802,Aventura,FL,33180.0,335000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20335-W-Cou...,MARMLS,A11634946,N,Y,25.963144,-80.138003
42,PAST SALE,2024-10-15,Condo/Co-op,6900 Bay Dr Unit 6B,Miami Beach,FL,33141.0,249000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6900-Bay...,MARMLS,A11632960,N,Y,25.854149,-80.127191
35,PAST SALE,2024-10-14,Condo/Co-op,13500 NE 3rd Ct #124,North Miami,FL,33161.0,145000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13500-NE...,MARMLS,A11626549,N,Y,25.899624,-80.192301
106,PAST SALE,2024-10-14,Condo/Co-op,900 Biscayne Blvd #4102,Miami,FL,33132.0,1200000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Biscayne-B...,MARMLS,A11529764,N,Y,25.783337,-80.190021


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

111


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$706,659


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$479


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$78,439,133


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
83
Input Previous Week Condo Average Sales Price:
937542
Input Previous Week Condo Average PSF:
561
Input Previous Week Condo Sales total (ex: 198_000_000)
77_800_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange
1,PAST SALE,2024-10-18,Condo/Co-op,16901 Collins Ave #1405,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,5.5,JADE SIGNATURE CONDO,3024.0,NaN,2018.0,NaN,1505.0,4661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11666904,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Fernando Villagran,Partnership Realty Inc.,None,None,Marcelo Violland,Prime International Associates Inc,None,None,2,blue
2,PAST SALE,2024-10-16,Condo/Co-op,9601 Collins Ave #710,Bal Harbour,FL,33154.0,3175000.0,3.0,3.5,MAJESTIC TOWER,2430.0,NaN,1998.0,NaN,1307.0,2587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11537819,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Helen Finvarb,"Finvarb Realty, Inc.",None,None,Alexandra Fournel,"New Wave Realty, LLC.",None,None,3,blue
3,PAST SALE,2024-10-15,Condo/Co-op,21200 Point Pl #1105,Aventura,FL,33180.0,2352000.0,5.0,5.5,ATLANTIC I AT THE POINT C,4171.0,NaN,2004.0,NaN,564.0,4680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21200-Point...,MARMLS,A11619174,N,Y,25.973121,-80.125125,https://www.redfin.com/FL/Aventura/21200-Point...,Natasha Gaguine,Coldwell Banker Realty,Claudia Whitechurch,Coldwell Banker Realty,Amir Hayun,Golani Realty Inc,None,None,4,blue
4,PAST SALE,2024-10-15,Condo/Co-op,801 S Miami Ave Unit LPH 5602,Miami,FL,33130.0,2300000.0,3.0,3.0,801 SMA RESIDENCES CONDO,2177.0,NaN,2018.0,NaN,1056.0,3547.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11612499,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Alexandar Alexandrov,"Compass Florida, LLC.",None,None,Matt Crane,"Compass Florida, LLC.",None,None,5,blue
5,PAST SALE,2024-10-17,Condo/Co-op,17111 Biscayne Blvd #2209,North Miami Beach,FL,33160.0,2295000.0,3.0,3.5,MARINA PALMS RESIDENCES S,2225.0,NaN,2017.0,NaN,1031.0,1695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17111-Bisca...,MARMLS,A11546582,N,Y,25.934152,-80.149117,https://www.redfin.com/FL/Aventura/17111-Bisca...,Alona Yona,Beachfront Realty Inc,None,None,Isabella Fraguio,Fortune Christie's International Real Estate,None,None,6,blue
6,PAST SALE,2024-10-17,Condo/Co-op,7000 Island Blvd #1607,Aventura,FL,33160.0,2100000.0,4.0,4.5,7000 ISL BLVD WILLIAMS IS,3180.0,NaN,1998.0,NaN,660.0,3179.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/7000-Island...,MARMLS,A11554498,N,Y,25.944165,-80.135467,https://www.redfin.com/FL/Aventura/7000-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue
7,PAST SALE,2024-10-15,Condo/Co-op,700 NE 26th Ter #2503,Miami,FL,33137.0,1880000.0,2.0,2.5,MISSONI BAIA,1732.0,NaN,2023.0,NaN,1085.0,2803.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11453436,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Diego Guillen,"DGS Real Estate, LLC",Andreina Espana,"DGS Real Estate, LLC",Jose Hernandez,One Sotheby's International Realty,None,None,8,blue
8,PAST SALE,2024-10-15,Condo/Co-op,450 Alton Rd #1910,Miami Beach,FL,33139.0,1680000.0,2.0,2.0,ICON CONDO,1370.0,NaN,2004.0

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_10212024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5101 in Miami'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5101 ...
1,PAST SALE,2024-10-18,Condo/Co-op,16901 Collins Ave #1405,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,5.5,JADE SIGNATURE CONDO,3024.0,NaN,2018.0,NaN,1505.0,4661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11666904,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Fernando Villagran,Partnership Realty Inc.,None,None,Marcelo Violland,Prime International Associates Inc,None,None,2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #140...
2,PAST SALE,2024-10-16,Condo/Co-op,9601 Collins Ave #710,Bal Harbour,FL,33154.0,3175000.0,3.0,3.5,MAJESTIC TOWER,2430.0,NaN,1998.0,NaN,1307.0,2587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11537819,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Helen Finvarb,"Finvarb Realty, Inc.",None,None,Alexandra Fournel,"New Wave Realty, LLC.",None,None,3,blue,MAJESTIC TOWER at 9601 Collins Ave #710 in Bal...
3,PAST SALE,2024-10-15,Condo/Co-op,21200 Point Pl #1105,Aventura,FL,33180.0,2352000.0,5.0,5.5,ATLANTIC I AT THE POINT C,4171.0,NaN,2004.0,NaN,564.0,4680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21200-Point...,MARMLS,A11619174,N,Y,25.973121,-80.125125,https://www.redfin.com/FL/Aventura/21200-Point...,Natasha Gaguine,Coldwell Banker Realty,Claudia Whitechurch,Coldwell Banker Realty,Amir Hayun,Golani Realty Inc,None,None,4,blue,ATLANTIC I AT THE POINT C at 21200 Point Pl #1...
4,PAST SALE,2024-10-15,Condo/Co-op,801 S Miami Ave Unit LPH 5602,Miami,FL,33130.0,2300000.0,3.0,3.0,801 SMA RESIDENCES CONDO,2177.0,NaN,2018.0,NaN,1056.0,3547.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11612499,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Alexandar Alexandrov,"Compass Florida, LLC.",None,None,Matt Crane,"Compass Florida, LLC.",None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Un...
5,PAST SALE,2024-10-17,Condo/Co-op,17111 Biscayne Blvd #2209,North Miami Beach,FL,33160.0,2295000.0,3.0,3.5,MARINA PALMS RESIDENCES S,2225.0,NaN,2017.0,NaN,1031.0,1695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17111-Bisca...,MARMLS,A11546582,N,Y,25.934152,-80.149117,https://www.redfin.com/FL/Aventura/17111-Bisca...,Alona Yona,Beachfront Realty Inc,None,None,Isabella Fraguio,Fortune Christie's International Real Estate,None,None,6,blue,MARINA PALMS RESIDENCES S at 17111 Biscayne Bl...
6,PAST SALE,2024-10-17,Condo/Co-op,7000 Island Blvd #1607,Aventura,FL,33160.0,2100000.0,4.0,4.5,7000 ISL BLVD WILLIAMS IS,3180.0,NaN,1998.0,NaN,660.0,3179.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/7000-Island...,MARMLS,A11554498,N,Y,25.944165,-80.135467,https://www.redfin.com/FL/Aventura/7000-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,7000 ISL BLVD WILLIAMS IS at 7000 Island Blvd ...
7,PAST SALE,2024-10-15,Condo/Co-op,700 NE 26th Ter #2503,Miami,FL,33137.0,1880000.0,2.0,2.5,MISSONI BAIA,1732.0,NaN,2023.0,NaN,1085.0,2803.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te..

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: BISCAYNE BEACH CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,600,000M to $8,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Miami Beach, Bal Harbour, Aventura, North Miami Beach, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $78,439,133 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 83 condo sales totaling $77,800,000.

Last week’s units sold for an average of $706,659, lower than the $937,542 average price from the previous w

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5101 ...
1,PAST SALE,2024-10-18,Condo/Co-op,16901 Collins Ave #1405,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,5.5,JADE SIGNATURE CONDO,3024.0,NaN,2018.0,NaN,1505.0,4661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11666904,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Fernando Villagran,Partnership Realty Inc.,None,None,Marcelo Violland,Prime International Associates Inc,None,None,2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #140...
2,PAST SALE,2024-10-16,Condo/Co-op,9601 Collins Ave #710,Bal Harbour,FL,33154.0,3175000.0,3.0,3.5,MAJESTIC TOWER,2430.0,NaN,1998.0,NaN,1307.0,2587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11537819,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Helen Finvarb,"Finvarb Realty, Inc.",None,None,Alexandra Fournel,"New Wave Realty, LLC.",None,None,3,blue,MAJESTIC TOWER at 9601 Collins Ave #710 in Bal...
3,PAST SALE,2024-10-15,Condo/Co-op,21200 Point Pl #1105,Aventura,FL,33180.0,2352000.0,5.0,5.5,ATLANTIC I AT THE POINT C,4171.0,NaN,2004.0,NaN,564.0,4680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21200-Point...,MARMLS,A11619174,N,Y,25.973121,-80.125125,https://www.redfin.com/FL/Aventura/21200-Point...,Natasha Gaguine,Coldwell Banker Realty,Claudia Whitechurch,Coldwell Banker Realty,Amir Hayun,Golani Realty Inc,None,None,4,blue,ATLANTIC I AT THE POINT C at 21200 Point Pl #1...
4,PAST SALE,2024-10-15,Condo/Co-op,801 S Miami Ave Unit LPH 5602,Miami,FL,33130.0,2300000.0,3.0,3.0,801 SMA RESIDENCES CONDO,2177.0,NaN,2018.0,NaN,1056.0,3547.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11612499,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Alexandar Alexandrov,"Compass Florida, LLC.",None,None,Matt Crane,"Compass Florida, LLC.",None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Un...
5,PAST SALE,2024-10-17,Condo/Co-op,17111 Biscayne Blvd #2209,North Miami Beach,FL,33160.0,2295000.0,3.0,3.5,MARINA PALMS RESIDENCES S,2225.0,NaN,2017.0,NaN,1031.0,1695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17111-Bisca...,MARMLS,A11546582,N,Y,25.934152,-80.149117,https://www.redfin.com/FL/Aventura/17111-Bisca...,Alona Yona,Beachfront Realty Inc,None,None,Isabella Fraguio,Fortune Christie's International Real Estate,None,None,6,blue,MARINA PALMS RESIDENCES S at 17111 Biscayne Bl...
6,PAST SALE,2024-10-17,Condo/Co-op,7000 Island Blvd #1607,Aventura,FL,33160.0,2100000.0,4.0,4.5,7000 ISL BLVD WILLIAMS IS,3180.0,NaN,1998.0,NaN,660.0,3179.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/7000-Island...,MARMLS,A11554498,N,Y,25.944165,-80.135467,https://www.redfin.com/FL/Aventura/7000-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,7000 ISL BLVD WILLIAMS IS at 7000 Island Blvd ...
7,PAST SALE,2024-10-15,Condo/Co-op,700 NE 26th Ter #2503,Miami,FL,33137.0,1880000.0,2.0,2.5,MISSONI BAIA,1732.0,NaN,2023.0,NaN,1085.0,2803.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te..

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: BISCAYNE BEACH CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,600,000M to $8,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami, Miami Beach, Bal Harbour, Aventura, North Miami Beach, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $78,439,133 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 83 condo sales totaling $77,800,000.

Last week’s units sold for an average of $706,659, lower than the $937,542 average price from the previous w

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/2900-NE-7th-Ave-33137/unit-5101/home/79132229


In [57]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-1405/home/56743903


In [59]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17555-Collins-Ave-33160/unit-1203/home/43244698


In [60]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/450-Alton-Rd-33139/unit-1910/home/43317909


In [61]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-1405/home/56743903


In [62]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/21200-Point-Pl-33180/unit-1105/home/191596694


In [63]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-15,Condo/Co-op,2900 NE 7th Ave #5101,Miami,FL,33137.0,8250000.0,5.0,6.5,BISCAYNE BEACH CONDO,7878.0,NaN,2017.0,NaN,1047.0,9427.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11563393,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bill Hernandez,Douglas Elliman,None,None,Daniel Hertzberg,Coldwell Banker Realty,None,None,1,orange,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #5101 ...
1,PAST SALE,2024-10-18,Condo/Co-op,16901 Collins Ave #1405,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,5.5,JADE SIGNATURE CONDO,3024.0,NaN,2018.0,NaN,1505.0,4661.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11666904,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Fernando Villagran,Partnership Realty Inc.,None,None,Marcelo Violland,Prime International Associates Inc,None,None,2,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #140...
2,PAST SALE,2024-10-16,Condo/Co-op,9601 Collins Ave #710,Bal Harbour,FL,33154.0,3175000.0,3.0,3.5,MAJESTIC TOWER,2430.0,NaN,1998.0,NaN,1307.0,2587.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11537819,N,Y,25.887706,-80.122180,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,Helen Finvarb,"Finvarb Realty, Inc.",None,None,Alexandra Fournel,"New Wave Realty, LLC.",None,None,3,blue,MAJESTIC TOWER at 9601 Collins Ave #710 in Bal...
3,PAST SALE,2024-10-15,Condo/Co-op,21200 Point Pl #1105,Aventura,FL,33180.0,2352000.0,5.0,5.5,ATLANTIC I AT THE POINT C,4171.0,NaN,2004.0,NaN,564.0,4680.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21200-Point...,MARMLS,A11619174,N,Y,25.973121,-80.125125,https://www.redfin.com/FL/Aventura/21200-Point...,Natasha Gaguine,Coldwell Banker Realty,Claudia Whitechurch,Coldwell Banker Realty,Amir Hayun,Golani Realty Inc,None,None,4,blue,ATLANTIC I AT THE POINT C at 21200 Point Pl #1...
4,PAST SALE,2024-10-15,Condo/Co-op,801 S Miami Ave Unit LPH 5602,Miami,FL,33130.0,2300000.0,3.0,3.0,801 SMA RESIDENCES CONDO,2177.0,NaN,2018.0,NaN,1056.0,3547.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,MARMLS,A11612499,N,Y,25.765990,-80.193097,https://www.redfin.com/FL/Miami/801-S-Miami-Av...,Alexandar Alexandrov,"Compass Florida, LLC.",None,None,Matt Crane,"Compass Florida, LLC.",None,None,5,blue,801 SMA RESIDENCES CONDO at 801 S Miami Ave Un...
5,PAST SALE,2024-10-17,Condo/Co-op,17111 Biscayne Blvd #2209,North Miami Beach,FL,33160.0,2295000.0,3.0,3.5,MARINA PALMS RESIDENCES S,2225.0,NaN,2017.0,NaN,1031.0,1695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17111-Bisca...,MARMLS,A11546582,N,Y,25.934152,-80.149117,https://www.redfin.com/FL/Aventura/17111-Bisca...,Alona Yona,Beachfront Realty Inc,None,None,Isabella Fraguio,Fortune Christie's International Real Estate,None,None,6,blue,MARINA PALMS RESIDENCES S at 17111 Biscayne Bl...
6,PAST SALE,2024-10-17,Condo/Co-op,7000 Island Blvd #1607,Aventura,FL,33160.0,2100000.0,4.0,4.5,7000 ISL BLVD WILLIAMS IS,3180.0,NaN,1998.0,NaN,660.0,3179.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/7000-Island...,MARMLS,A11554498,N,Y,25.944165,-80.135467,https://www.redfin.com/FL/Aventura/7000-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,7,blue,7000 ISL BLVD WILLIAMS IS at 7000 Island Blvd ...
7,PAST SALE,2024-10-15,Condo/Co-op,700 NE 26th Ter #2503,Miami,FL,33137.0,1880000.0,2.0,2.5,MISSONI BAIA,1732.0,NaN,2023.0,NaN,1085.0,2803.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te..

## Time on Market Calculator

In [58]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 26) ## List (Earlier) date
date2 = datetime(2024, 10, 18) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

22


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [72]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")